In [62]:
import pandas as pd
import os
import json
import re
import operator
import json
from os.path import join
pd.set_option('display.max_colwidth', None)
from examples.hitting_sets.increasing_hyp_space import extra_configs

In [92]:
base_table = r'''
\begin{table}[H]
\centering
\resizebox{0.8\textwidth}{!}{%
\begin{tabular}{cccc}
\toprule
& \textbf{\makecell{Domain\\knowledge}} & \textbf{\makecell{Convergence\\iteration}} & \textbf{\makecell{Convergence\\time (s)}} \\
\midrule
<<NSL_RESULTS_TO_REPLACE>>
\bottomrule
\end{tabular}
}
\caption{Increasing the hypothesis space, HS FashionMNIST task.}
\label{tab:hs_increasing_hyp_space}
\end{table}
'''

In [93]:
def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

In [94]:
example_dir = '../../../examples/hitting_sets'
NSL_BASE_RUN_DIR = example_dir+'/saved_results/saved_local_runs/run_1'
NSL_3_1_RUN_DIR = example_dir+'/saved_results/repeats/HS_fashion_mnist/100/1'

In [95]:
def get_mode_declarations():
    config_results = {}
    for c in extra_configs:
        config_results[c] = extra_configs[c].md.split('#bias(":- in_head(H1), in_head(H2), H1<H2.").\n')[1].split('\n')
    return config_results

In [96]:
def get_convergence(run_dir):
    with open(join(run_dir, 'test_log.json'), 'r') as jf:
        tl = json.loads(jf.read())
        for e in tl:
            if tl[str(e)]['hyp_accuracy'] >= 1:
                convergence_epoch = int(e)
                break
    if convergence_epoch == 0:
        convergence_epoch = 1
        
    train_details_file = run_dir+'/train_details.txt'
    iteration_times = []
    read=False
    for line in open(train_details_file):
        if '"total_without_test"' in line.strip():
            val = line.strip().split(': ')[1]
            iteration_times.append(float(val))
            read = True
            continue
        elif line.strip() == '---------------------':
            read = False
    # Sum how long it takes to reach this epoch
    epoch_time = 0
    for j in range(int(convergence_epoch)):
        epoch_time += iteration_times[j]
    return {"convergence_epoch": convergence_epoch, "time": epoch_time}

In [97]:
# Generate convergence iteration and run time until convergence automatically
def get_convergence_iteration_and_run_time(example_dir):
    nsl_dir = example_dir+'/saved_results/increasing_hyp_space/HS_fashion_MNIST/100'
    configs = os.listdir(nsl_dir)
    configs = [c for c in configs if c != '.DS_Store']
    configs = [c for c in configs if c != '.DS_Store']
    configs = [c for c in configs if c != '__init__']
    configs = [c for c in configs if c != '__init__.py']
    configs = [c for c in configs if c != '__pycache__']
    configs = [c for c in configs if c != 'keep.txt']
    configs = [c for c in configs if '.json' not in c]
    
    config_results = {}
    
    # Get convergence and time of base
    config_results['BASE'] = get_convergence(NSL_BASE_RUN_DIR)

    # Sampled configs
    for c in configs:
        config_results[c] = get_convergence(nsl_dir+'/'+c)
        
    # 3 sets digit 1
    config_results['FINAL'] = get_convergence(NSL_3_1_RUN_DIR)
    
    return config_results

In [98]:
def create_table_row_str(data_list):
    table_row_str = ''
    for r in data_list:
        for idx, cell in enumerate(r):
            if idx == 3:
                table_row_str += '{:.2f}'.format(cell) + ' & '
            else:
                table_row_str += str(cell) +' & '
        table_row_str = table_row_str[:-2] + r'\\ \midrule'
    table_row_str = table_row_str[:-9]
    return table_row_str

In [99]:
def create_hyp_space_table(example_dir, caption):
    bk_ss = get_mode_declarations()
    iter_run_time = get_convergence_iteration_and_run_time(example_dir)
    nsl_dir = example_dir+'/saved_results/increasing_hyp_space/HS_fashion_mnist/100'
    data = []
    configs = os.listdir(nsl_dir)
    configs = [c for c in configs if c != '.DS_Store']
    configs = [c for c in configs if c != '__init__']
    configs = [c for c in configs if c != '__init__.py']
    configs = [c for c in configs if c != '__pycache__']
    configs = [c for c in configs if '.json' not in c]
    configs = [c for c in configs if 'keep.txt' not in c]
    configs.sort(key=natural_keys)
    
    # Add standard
    data.append(['', 
                 r'Standard', 
                iter_run_time['BASE']['convergence_epoch'], 
                round(iter_run_time['BASE']['time'],2)])
    
    for c in configs:
        # Ignore config 2 as basically the same as config 1
        if c != 'config_2':
            mode_decs_for_display = ['Standard']
            for m in sorted(bk_ss[c]):
                if 'var(ss)' in m:
                    # means element
                    el = m.split('var(ss), ')[1].split('),')[0]
                    mode_decs_for_display.append('el {0}'.format(el))
                elif 'var(elt)' in m:
                    # means subset
                    ss_id = m.split(', var(elt)')[0].split('ss_element(')[1]
                    mode_decs_for_display.append('ssID {0}'.format(ss_id))
            mode_decs_for_display = ', '.join(mode_decs_for_display)
            data.append([
                '',
                mode_decs_for_display,
                int(iter_run_time[c]['convergence_epoch']), 
                round(iter_run_time[c]['time'],2)
            ])
        
    
    # Add 3 sets digit 1
    md = 'Standard, ssID 3, el 1'
    data.append(['*', 
                 md, 
                 iter_run_time['FINAL']['convergence_epoch'],
                 round(iter_run_time['FINAL']['time'],2)
                ])
    
    # Sort by convergence epoch then time
    data = sorted(data, key=operator.itemgetter(2, 3))

    table_row_str = create_table_row_str(data)
    table_str = base_table.replace('<<NSL_RESULTS_TO_REPLACE>>', table_row_str)
    
    return table_str

In [100]:
print(create_hyp_space_table(example_dir, 'Sampled hypothesis spaces - Hitting Sets standard task. Bold shows configuration used in this paper.'))


\begin{table}[H]
\centering
\resizebox{0.8\textwidth}{!}{%
\begin{tabular}{cccc}
\toprule
& \textbf{\makecell{Domain\\knowledge}} & \textbf{\makecell{Convergence\\iteration}} & \textbf{\makecell{Convergence\\time (s)}} \\
\midrule
 & Standard, ssID 4, el 4 & 1 & 97.64 \\ \midrule & Standard, ssID 4, el 3 & 1 & 102.16 \\ \midrule & Standard, el 3, el 4 & 1 & 110.80 \\ \midrule & Standard, ssID 2, ssID 4 & 1 & 111.34 \\ \midrule & Standard, ssID 2, el 2 & 1 & 113.21 \\ \midrule & Standard & 1 & 115.77 \\ \midrule & Standard, ssID 1, el 4 & 1 & 130.71 \\ \midrule & Standard, ssID 3, el 4 & 2 & 734.70 \\ \midrule* & Standard, ssID 3, el 1 & 2 & 852.84 \\ \midrule & Standard, ssID 3, el 3 & 2 & 864.03 \\ \midrule & Standard, ssID 1, ssID 3 & 2 & 888.54 \\
\bottomrule
\end{tabular}
}
\caption{Increasing the hypothesis space, HS FashionMNIST task.}
\label{tab:hs_increasing_hyp_space}
\end{table}

